This notebook uses an sgcn module in the bispy package to search the National List SGCN API. It returns and caches the summarized National List records, which include the list of states that have the species in their conservation planning process.

In [1]:
#Import needed packages
import bispy
import json
from IPython.display import display
from joblib import Parallel, delayed

sgcn = bispy.sgcn.Search()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached updated ITIS list with valid/accepted species names
with open("cache/updated_itis.json", "r") as f:
    valid_itis = json.loads(f.read())

In [3]:
valid_itis

[{'scientific_name': 'Festuca idahoensis', 'itis_usage': 'accepted'},
 {'scientific_name': 'Ursus americanus', 'itis_usage': 'valid'},
 {'scientific_name': 'Pseudotsuga menziesii', 'itis_usage': 'accepted'},
 {'scientific_name': 'Tamiasciurus hudsonicus', 'itis_usage': 'valid'},
 {'scientific_name': 'Otis tarda', 'itis_usage': 'valid'},
 {'scientific_name': 'Melanitta perspicillata', 'itis_usage': 'valid'},
 {'scientific_name': 'Taxidea taxus', 'itis_usage': 'valid'},
 {'scientific_name': 'Puccinellia rupestris', 'itis_usage': 'accepted'},
 {'scientific_name': 'Artemisiospiza belli', 'itis_usage': 'valid'},
 {'scientific_name': 'Rangifer tarandus groenlandicus', 'itis_usage': 'valid'},
 {'scientific_name': 'Urocitellus armatus', 'itis_usage': 'valid'},
 {'scientific_name': 'Centrocercus urophasianus', 'itis_usage': 'valid'},
 {'scientific_name': 'Procapra gutturosa', 'itis_usage': 'valid'},
 {'scientific_name': 'Juniperus communis', 'itis_usage': 'accepted'},
 {'scientific_name': 'Cron

In [5]:
# Use joblib to run multiple requests for TESS documents in parallel via the valid/accepted ITIS scientific name
sgcn_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in valid_itis]) 
len(sgcn_result)

116

In [6]:
# Filter to give just cases where a SGCN species name matched with the valid/accepted ITIS names
sgcn_success=[i for i in sgcn_result if i['processing_metadata']['status'] == 'success']
len(sgcn_success)

57

In [7]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/sgcn.json", sgcn_success))

{'Doc Cache File': 'cache/sgcn.json',
 'Number of Documents in Cache': 57,
 'Document Number 38': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-06T21:48:28.377184',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Alces alces'},
  'parameters': {'Scientific Name': 'Alces alces', 'Name Source': None},
  'sgcn_species': {'statelist_2005': 'Massachusetts,Michigan,Wisconsin,Wyoming',
   'statelist_2015': 'Vermont,Wyoming',
   'scientificname': 'Alces alces',
   'commonname': 'moose',
   'taxonomicgroup': 'Mammals',
   'taxonomicrank': 'Species',
   'matchmethod': 'Exact Match',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=180703'}}}

Here we look to see which WLCI species show up on the Wyoming SGCN list. 

In [15]:
sgcn_wy_names = []
for spp in [r["sgcn_species"] for r in sgcn_success if r["processing_metadata"]["status"] == "success"]:
    if "Wyoming" in spp["statelist_2005"] or "Wyoming" in spp["statelist_2015"]:
        print(spp["commonname"], '======', spp["scientificname"])
        sgcn_wy_names.append(spp["scientificname"])

Bell's Sparrow ====== Artemisiospiza belli
Uinta Ground Squirrel ====== Urocitellus armatus
Greater Sage Grouse ====== Centrocercus urophasianus
Brewer's Sparrow ====== Spizella breweri
cutthroat trout ====== Oncorhynchus clarkii
Ferruginous Hawk ====== Buteo regalis
mottled sculpin ====== Cottus bairdii
Bald Eagle ====== Haliaeetus leucocephalus
Sage Thrasher ====== Oreoscoptes montanus
mountain sucker ====== Catostomus platyrhynchus
moose ====== Alces alces
Colorado River cutthroat trout ====== Oncorhynchus clarkii pleuriticus
bighorn sheep ====== Ovis canadensis
Pygmy Rabbit ====== Brachylagus idahoensis
Brown Bear ====== Ursus arctos


In [16]:
len(sgcn_wy_names)

15

Note

There were no SGCN records found based on the names that ITIS considered invalid or not accepted

In [8]:
# Open up the cached updated ITIS list with invalid/not accepted species names
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [9]:
# Use joblib to run multiple requests for SGCN documents in parallel via the invalid/not accepted ITIS scientific names
invalid_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [10]:
# Filter to give just cases where a SGCN species name matched with the invalid/not accepted ITIS names
invalid_result=[i for i in invalid_result if i['processing_metadata']['status'] == 'success']

In [11]:
len(invalid_result)

0